In [3]:
import tensorflow as tf
import cv2
import numpy as np
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization, Input,  MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.applications import VGG16, InceptionResNetV2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model, Model
from keras import regularizers
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd /content/drive/MyDrive/Emotion

/content/drive/MyDrive/Emotion


In [6]:
# !unzip Emotion2.zip
#la base de donnée contient 7 classes mais on a utilisé seulement 3 classes

In [7]:
ls

best_model.h5  Emotion.zip  sad.jpg  test/  train/  vgg16_tfds.h5


In [8]:

img_size = 48

In [9]:

train_datagen = ImageDataGenerator(#rotation_range = 180,
                                         width_shift_range = 0.1,
                                         height_shift_range = 0.1,
                                         horizontal_flip = True,
                                         rescale = 1./255,
                                         #zoom_range = 0.2,
                                         validation_split = 0.2
                                        )
validation_datagen = ImageDataGenerator(rescale = 1./255,
                                         validation_split = 0.2)

In [11]:

train_generator = train_datagen.flow_from_directory(
        'train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        'test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 17010 images belonging to 3 classes.
Found 4254 images belonging to 3 classes.


In [12]:
#l'architecture utilisé
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(3, activation='softmax'))

In [14]:
epochs = 60
batch_size = 64

In [15]:
emotion_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 128)      0         
 2D)                                                    

In [16]:
# Simple early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=8)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [17]:
emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [18]:
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=17010 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=4254 // 64)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/50
265/265 [==============================] - 3159s 12s/step - loss: 1.0766 - accuracy: 0.4235 - val_loss: 1.0666 - val_accuracy: 0.4356
Epoch 2/50
265/265 [==============================] - 34s 127ms/step - loss: 1.0308 - accuracy: 0.4740 - val_loss: 0.9526 - val_accuracy: 0.5597
Epoch 3/50
265/265 [==============================] - 34s 127ms/step - loss: 0.9769 - accuracy: 0.5279 - val_loss: 0.8802 - val_accuracy: 0.5926
Epoch 4/50
265/265 [==============================] - 34s 128ms/step - loss: 0.9268 - accuracy: 0.5623 - val_loss: 0.8197 - val_accuracy: 0.6165
Epoch 5/50
265/265 [==============================] - 33s 126ms/step - loss: 0.8766 - accuracy: 0.5935 - val_loss: 0.7805 - val_accuracy: 0.6484
Epoch 6/50
265/265 [==============================] - 34s 128ms/step - loss: 0.8473 - accuracy: 0.6101 - val_loss: 0.7585 - val_accuracy: 0.6446
Epoch 7/50
265/265 [==============================] - 33s 125ms/step - loss: 0.8080 - accuracy: 0.6276 - val_loss: 0.7434 - val_ac

In [19]:
# save model structure in jason file
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model.h5')